# RNN pour l'analyse de sentiments

Nous allons maintenant faire une analyse de sentiments sur le même jeu de données en utilisant les réseaux de neurones récurrents

## Réseaux de neurones récurrents

Les réseaux de neurones récurrents ou (RNN), sont souvent utilisés pour analyser des séquences.

En effet, dans les réseaux de neurones généraux, un input est traité par un certain nombre de couches et un output est produit à la sortie, avec l'hypothèse que deux inputs successifs sont indépendants.

Cependant, cette hypothèse n'est pas correcte dans un certain nombre de scénarios. 
Par exemple, si on veut prédire le mot suivant dans une séquence, il est indispensable de considérer la dépendance des observations précédentes.
    
Dans notre cas, le modèle RNN prend une séquence de mots $X=\{x_1, ..., x_T\}$, une à la fois, et produit un état caché $h$, pour chaque mot.
On utilise le RNN en lui donnant le mot courant $x_t$ ainsi que l'état caché du mot précédent, $h_{t-1}$, pour produire l'état caché suivant, $h_t$.


$$h_t = \text{RNN}(x_t, h_{t-1})$$


Une fois que l'on a notre état caché final, $h_T$, obtenu après avoir donné le dernier mot de la séquence $x_T$ au modèle, on le donne à une couche linéaire $f$, (qui s'appelle également fully connected layer), pour recevoir notre sentiment prédit, $\hat{y} = f(h_T)$.

<center> <img src="RNN.png" alt="drawing" width="700"/>
        
        
Cette illustration montre un exemple de phrase, avec le RNN prédisant 0, c'est-à-dire que le sentiment est négatif. Le RNN est représenté en orange et la couche linéaire est en gris. On utilise le même RNN pour chaque mot, c'est-à-dire qu'il a les mêmes paramètres. L'état initial caché $h_0$, est un tensor initialisé à zéro.

## Préparation des données

Dans ce notebook, on utilise la librairie torch et TorchText.

TorchText a une méthode `Field` qui sert à définir comment les données brutes doivent être traitées.

La méthode `TEXT` définit comment les commentaires doivent être traités, et `LABEL` comment les labels doivent être traités. 

`TEXT` a un argument `tokenize`, qui par défaut split les chaînes de caractères en espaces.

In [1]:
import pandas as pd
import torch
from torchtext import data

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


TEXT = data.Field(sequential=True,lower=True, tokenize = 'spacy')

LABEL = data.LabelField(dtype = torch.float)

On sépare les données en train et test.

In [2]:
train_data, valid_data, test_data = data.TabularDataset.splits(
        path='./data/', train='train.csv',
        validation='valid.csv', test='test.csv', format='csv', skip_header=True,
        fields=[('text', TEXT), ('label', LABEL)])

print(f'Taille des données train: {len(train_data)}')
print(f'Taille des données de validation: {len(valid_data)}')
print(f'Taille des données test: {len(test_data)}')

Taille des données train: 18163
Taille des données de validation: 2270
Taille des données test: 2271


On affiche un exemple.

In [3]:
print(vars(train_data.examples[0]))

{'text': ['this', 'movie', 'is', 'a', 'gem', 'because', 'it', 'moves', 'with', 'soft', ',', 'but', 'firm', 'resolution.<br', '/><br', '/>i', 'caution', 'viewers', 'that', 'although', 'it', 'is', 'billed', 'as', 'a', 'corporate', 'spy', 'thriller', 'and', 'ms', 'liu', 'is', 'there', ',', 'it', 'moves', 'at', 'a', 'deftly', 'purposeful', 'yet', 'sedate', 'pace', '.', 'it', "'s", 'not', 'about', 'explosions', ',', 'car', 'chases', ',', 'or', 'flying', 'bullets', '.', 'you', 'must', 'be', 'patient', 'and', 'instead', ',', 'note', 'the', 'details', 'here', '.', 'it', "'s", 'sedate', 'because', 'that', "'s", 'what', 'the', 'main', 'character', 'is', '.', 'the', 'viewer', 'has', 'to', 'watch', 'him', 'and', 'think', 'as', 'this', 'story', 'unfolds.<br', '/><br', '/>i', 'will', 'not', 'give', 'spoilers--', 'because', 'that', 'destroys', 'the', 'point', 'of', 'watching', '.', 'the', 'plot', 'is', 'what', 'you', "'ve", 'read', 'from', 'the', 'other', 'postings', ':', 'an', 'average', 'white', '-

On crée un échantillon de données de validation.

On crée un vocabulaire sur l'échantillon train avec la méthode `build_vocab`et on ne prend que 25000 mots pour diminuer la dimension de la matrice d'embedding.

In [4]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(f"Nombre de tokens unique dans le TEXT: {len(TEXT.vocab)}") 
print(f"Nombre unique de LABEL: {len(LABEL.vocab)}")

Nombre de tokens unique dans le TEXT: 10002
Nombre unique de LABEL: 2


Les deux valeurs supplémentaires dans le vocabulaire du TEXT sont les tokens `<unk>` et `<pad>`.


In [5]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 274223), (',', 239861), ('.', 204258), ('and', 135511), ('a', 135466), ('of', 121892), ('to', 112826), ('is', 92798), ('in', 77926), ('it', 74198), ('i', 60943), ('that', 60269), ('this', 55592), ('"', 54672), ("'s", 54540), ('-', 45599), ('\n', 42455), ('as', 39521), ('with', 37130), ('was', 36791)]


Les valeurs du vocabulaire de LABEL sont 0 pour positif et 1 pour négatif.

In [6]:
print(LABEL.vocab.stoi)

defaultdict(None, {'0': 0, '1': 1})


La dernière étape de la préparation des données consiste à créer les itérateurs. Nous les parcourons dans la boucle d'apprentissage / d'évaluation, et ils retournent un lot d'exemples (indexés et convertis en tenseurs) à chaque itération.

On utilise `BucketIterator` qui est un itérateur qui renverra un lot d'exemples où chaque exemple est d'une longueur similaire, minimisant la quantité de padding par exemple.

In [7]:
# utilisation du GPU si possible 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device, sort = False)

## Construction du modèle

Le modèle RNN utilisé se compose des couches suivantes : 

- _embedding_ : utilisé pour transformer notre vecteur one hot encoder (dont la plupart des éléments sont des 0) en un vecteur embedding dense (dense car la dimensionnalité est beaucoup plus petite et tous les éléments sont des nombres réels). De plus, les mots qui ont un impact similaire sur le sentiment de la revue sont mappés de manière rapprochée dans cet espace vectoriel dense.
- _RNN_ : prend le vecteur dense et le précédent état caché $h_{t-1}$, et calcule l'état caché suivant $h_t$.
- _linéaire_ : prend le dernier état caché, le met dans un couche fully connected $f(h_T)$ qui le transfome en output prédit.

On appelle la méthode `forward` lorsque l'on donne nos exemple à notre modèle.

Chaque batch, `text` est un tensor de dimension _**[sentence length, batch size]**_. C'est un batch de commentaires, dont les mots sont convertis en représentation one-hot encoder. PyTorch stocke  un vecteur one-hot comme sa valeur d'index.

Le batch de l'input est ensuite passé à travers la couche embedding pour obtenir une représentation de vecteur dense de nos commentaires.
`embedded` est un tensor de taille _**[sentence length, batch size, embedding dim]**_.

`embedded`est ensuite introduit dans la couche RNN. Dans certains frameworks, vous devez alimenter l'état caché initial, $h_0$, dans le RNN, cependant dans PyTorch, si aucun état caché initial n'est passé comme argument, il prend par défaut un tenseur de valeurs zéros.

The RNN retourne 2 tenseurs, `output` de taille _**[sentence length, batch size, hidden dim]**_ et `hidden` de taille _**[1, batch size, hidden dim]**_.

`output` est la concaténation de l'état caché de chaque pas de temps, alors que `hidden` est l'état caché final. 
La méthode `squeeze` qui est utilisé pour supprimer une dimension de taille 1. 

Enfin, nous alimentons le dernier état caché, `hidden`, à travers la couche linéaire, `fc`, pour produire une prédiction.

In [8]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim,bidirectional):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim, bidirectional=bidirectional)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, text):

        embedded = self.embedding(text)
               
        output, hidden = self.rnn(embedded)
        
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        
        return self.fc(hidden)


On crée alors notre modèle.

La dimension de l'input est la taille du vocabulaire. 

La dimension de l'embedding est la taille des vecteurs denses. Elle est généralement de 50 à 250, mais elle dépend de la taille du vocabulaire.

La dimension de hidden est la dimension des états cachés. Elle vaut généralement 100 à 500, mais dépend de la taille du vocabulaire, des vecteurs denses et la compléxité de la tâche. 

La dimension de l'output est le nombre de classes, ici, la valeur de l'output est entre 0 et 1 et donc est de dimension 1 car l'output est un scalaire.

In [9]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
BIDIRECTIONAL = True
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,BIDIRECTIONAL )

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Le modèle a {count_parameters(model):,} paramètres à entraîner')

Le modèle a 1,184,009 paramètres à entraîner


## Entraîner le modèle

On entraîne le modèle.
On utilise l'optimiseur Adam qui ne prend en compte seulement comme paramètres les paramètres du modèle qui seront actualisés par l'optimiseur.

In [11]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

On définit ensuite la fonction de perte qui est ici la _binary cross entropy with logits_. 

Notre modèle génère actuellement un nombre réel non lié. Comme nos étiquettes sont 0 ou 1, nous voulons limiter les prédictions à un nombre entre 0 et 1. Nous le faisons en utilisant les fonctions _sigmoid_ ou _logit_.

Nous utilisons ensuite ce scalaire lié pour calculer la perte en utilisant l'entropie croisée binaire.

Le critère `BCEWithLogitsLoss` effectue à la fois les étapes d'entropie croisée sigmoïde et binaire.

In [12]:
criterion = nn.BCEWithLogitsLoss()

En utilisant `.to`, nous pouvons placer le modèle et le critère sur le GPU (si nous en avons un).

In [13]:
model = model.to(device)
criterion = criterion.to(device)

Notre fonction de critère calcule la perte, cependant nous devons écrire notre fonction pour calculer l'accuracy.

Cette fonction alimente d'abord les prédictions à travers une couche sigmoïde, en écrasant les valeurs entre 0 et 1, nous les arrondissons ensuite à l'entier le plus proche. Cela arrondit toute valeur supérieure à 0,5 à 1 (un sentiment positif) et le reste à 0 (un sentiment négatif).

Nous calculons ensuite le nombre de prédictions arrondies égales aux labels réels et la moyenne sur l'ensemble du batch.

In [14]:
def binary_accuracy(preds, y):
    """
    Retourne l'accuracy par batch
    """
    #arrondi la prédiction à l'entier le plus proche
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

def precision(preds, y):
    '''
    Retourne la précision
    '''
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float() 
            
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
       
    precision = tp / (tp + fp)
    
    return precision

def recall(preds, y):
    '''
    Retourne le recall
    '''
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float()       
    
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    recall = tp / (tp + fn)
    return recall


def f1_loss(preds, y):
    '''
    Retourne le score F1
    '''  
    y_pred = torch.round(torch.sigmoid(preds))
    y_true = (y_pred == y).float() 
            
    tp = (y_true * y_pred).sum().float()
    tn = ((1 - y_true) * (1 - y_pred)).sum().float()
    fp = ((1 - y_true) * y_pred).sum().float()
    fn = (y_true * (1 - y_pred)).sum().float()
    
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    
    f1 = 2* (precision*recall) / (precision + recall)
    return f1

La fonction `train` itère sur tous les exemples, un batch à la fois.

`model.train()`est utilisé pour mettre le modèle en "mode entraînement", qui active _dropout_ et _batch normalization_.

Pour chaque batch, on met à zéro le gradient. Chaque paramètre dans un modèle a un attribut `grad` qui stocke le gradient calculé par `criterion`. PyTorch ne met pas à zéro les gradients automatiquement du dernier calcul de graients. 

Nous introduisons ensuite le lot de phrases, `batch.text`, dans le modèle. Le `squeeze` est nécessaire car les prédictions sont initialement de taille _**[batch size, 1]**_, et nous devons supprimer la dimension de taille 1 car PyTorch s'attend à ce que les prédictions entrées dans notre fonction de critère soient de taille _**[batch size]**_.

La perte et la précision sont ensuite calculées à l'aide de nos prédictions et des labels, `batch.label`, la perte étant moyennée sur tous les exemples du batch.

Nous calculons le gradient de chaque paramètre avec `loss.backward ()`, puis mettons à jour les paramètres en utilisant les gradients et l'algorithme d'optimisation avec `optimizer.step ()`.

La perte et la précision s'accumulent à travers l'epoch, la méthode `.item ()` est utilisée pour extraire un scalaire d'un tenseur qui ne contient qu'une seule valeur.

Enfin, nous retournons la perte et la précision, moyennées sur toute l'époque. Le `len` d'un itérateur est le nombre de batchs dans l'itérateur.


In [15]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_prec = 0
    epoch_rec = 0
    epoch_f1 = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
               
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        prec = precision(predictions, batch.label)
        rec = recall(predictions, batch.label)
        f1 = f1_loss(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        epoch_prec += prec.item()
        epoch_rec += rec.item()
        epoch_f1 += f1.item()
        
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_prec / len(iterator), epoch_rec / len(iterator), epoch_f1 / len(iterator)

La fonction `evaluate` est similaire à` train`, avec quelques modifications car vous ne voulez pas mettre à jour les paramètres lors de l'évaluation.

`model.eval ()` met le modèle en "mode d'évaluation", ceci désactive _dropout_ et _batch normalization_.

Aucun gradient n'est calculé sur les opérations PyTorch à l'intérieur du bloc `with no_grad ()`. Cela réduit l'utilisation de la mémoire et accélère le calcul.

Le reste de la fonction est identique à `train`, avec la suppression de` optimizer.zero_grad () `,` loss.backward () `et` optimizer.step () `, car nous ne mettons pas à jour les paramètres du modèle lorsque évaluer.

In [16]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model.forward(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Nous entraînons ensuite le modèle à travers plusieurs époques, une époque étant un passage complet à travers tous les exemples dans les ensembles d'apprentissage et de validation.

À chaque époque, si la perte de validation est la meilleure que nous ayons vue jusqu'à présent, nous enregistrerons les paramètres du modèle, puis une fois l'entraînement terminé, nous utiliserons ce modèle sur l'ensemble de test.

In [17]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc, train_prec, train_rec, train_f1 = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%| Train Precision: {train_prec*100:.2f}%| Train Recall: {train_rec*100:.2f}%| Train F1: {train_f1*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

RuntimeError: CUDA out of memory. Tried to allocate 506.00 MiB (GPU 0; 15.90 GiB total capacity; 575.19 MiB already allocated; 215.56 MiB free; 900.00 MiB reserved in total by PyTorch)

## Tester le modèle

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Notre fonction predict_sentiment fait plusieurs choses:

 - définit le modèle en mode d'évaluation
 - tokenise la phrase, c'est-à-dire la divise d'une chaîne brute en une liste de jetons
 - indexe les jetons en les convertissant en leur représentation entière à partir de notre vocabulaire
 - obtient la longueur de notre séquence
 - convertit les index, qui sont une liste Python en un tenseur PyTorch
 - ajouter une dimension de lot en relâchant
 - convertit la longueur en un tenseur
 - écrase la prédiction de sortie d'un nombre réel compris entre 0 et 1 avec la fonction sigmoïde
 - convertit le tenseur contenant une valeur unique en un entier avec la méthode item ()

Nous nous attendons à ce que les avis avec un sentiment négatif renvoient une valeur proche de 0 et les avis positifs renvoient une valeur proche de 1.

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    print(tokenized)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    print(indexed)
    length = [len(indexed)]
    print(length)
    tensor = torch.LongTensor(indexed).to(device)
    print(tensor)
    tensor = tensor.unsqueeze(1)
    print(tensor)
    length_tensor = torch.LongTensor(length)
    print(length_tensor)
    prediction = torch.sigmoid(model(tensor))
    print(prediction)
    return prediction.item()

In [ ]:
predict_sentiment(model, "This film is terrible")

In [ ]:
predict_sentiment(model, "This film is great")